In [3]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def wnba_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/wnba/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
    datenum = list(map(lambda x: x[:10].replace("-",""),txt))
    date = list(map(lambda x: x[:10],txt))
    
    year = list(map(lambda x: x[:4],txt))
    month = list(map(lambda x: x[5:7],txt))
    day = list(map(lambda x: x[8:10],txt))
    
    data = {"season": season,
            "datetime" : txt,
            "date" : date,
            "year": year,
            "month": month,
            "day": day,
            "dateURL": datenum
            
           }
    df = pd.DataFrame(data)
    df['url']="http://site.api.espn.com/apis/site/v2/sports/basketball/wnba/scoreboard?dates="
    df['url']= df['url'] + df['dateURL']
    return df

wnba_calendar(2018)

,season,datetime,date,year,month,day,dateURL,url
0,2018,2018-05-06T07:00Z,2018-05-06,2018,05,06,20180506,http://site.api.espn.com/apis/site/v2/sports/b...
1,2018,2018-05-07T07:00Z,2018-05-07,2018,05,07,20180507,http://site.api.espn.com/apis/site/v2/sports/b...
2,2018,2018-05-08T07:00Z,2018-05-08,2018,05,08,20180508,http://site.api.espn.com/apis/site/v2/sports/b...
3,2018,2018-05-11T07:00Z,2018-05-11,2018,05,11,20180511,http://site.api.espn.com/apis/site/v2/sports/b...
4,2018,2018-05-12T07:00Z,2018-05-12,2018,05,12,20180512,http://site.api.espn.com/apis/site/v2/sports/b...
...,...,...,...,...,...,...,...,...
84,2018,2018-09-02T07:00Z,2018-09-02,2018,09,02,20180902,http://site.api.espn.com/apis/site/v2/sports/b...
85,2018,2018-09-04T07:00Z,2018-09-04,2018,09,04,20180904,http://site.api.espn.com/apis/site/v2/sports/b...
86,2018,2018-09-07T07:00Z,2018-09-07,2018,09,07,20180907,http://site.api.espn.com/apis/site/v2/sports/b...
87,2018,2018-09-09T07:00Z,2018-09-09,2018,09,09,20180909,http://site.api.espn.com/apis/site/v2/sports/b...


In [4]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = wnba_calendar(x)
    full_schedule.to_json(orient='records')
    schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('wnba_calendar.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

In [8]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time
def wnba_schedule(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/wnba/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
#     print(len(txt))
    txt = list(map(lambda x: x[:10].replace("-",""),txt))
    ev = pd.DataFrame()
    i=0
    for date in txt:
        print(f"Working on {season}: {i+1} of {len(txt)}, date: {txt[i]}")
        url = "http://site.api.espn.com/apis/site/v2/sports/basketball/wnba/scoreboard?dates={}&limit=1000".format(date)
        resp = download(url=url)
    
        if resp is not None:
            events_txt = json.loads(resp)

            events = events_txt['events']
            for event in events:
                bad_keys = ['linescores', 'statistics', 'leaders',  'records']
                for k in bad_keys:
                    if k in event['competitions'][0]['competitors'][0].keys():
                        del event['competitions'][0]['competitors'][0][k]
                    if k in event['competitions'][0]['competitors'][1].keys():
                        del event['competitions'][0]['competitors'][1][k]
                if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                    del event['competitions'][0]['competitors'][0]['team']['links']
                if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                    del event['competitions'][0]['competitors'][1]['team']['links']    
                if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
                else: 
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
                if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
                else: 
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

                del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
                for k in del_keys:
                    if k in event['competitions'][0].keys():
                        del event['competitions'][0][k]

                ev = ev.append(pd.json_normalize(event['competitions'][0]))
            i+=1
            ev['season']=season
        else:
            i+=1
    return ev


In [6]:
schedule_table = pd.DataFrame()
schedule = {}
for x in range(2002,2022):
    year_schedule = wnba_schedule(x)
    year_schedule.to_csv(f"wnba/schedules/wnba_games_info_{x}.csv")
    schedule_table = schedule_table.append(year_schedule)
    year_schedule.to_json(orient='records')
    schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}



schedule_table.to_csv('wnba_games_info_2002_2021.csv')
    
# event['competitions'][0]

Working on 2002: 1 of 94, date: 20020509
Working on 2002: 2 of 94, date: 20020510
Working on 2002: 3 of 94, date: 20020511
Working on 2002: 4 of 94, date: 20020512
Working on 2002: 5 of 94, date: 20020513
Working on 2002: 6 of 94, date: 20020515
Working on 2002: 7 of 94, date: 20020517
Working on 2002: 8 of 94, date: 20020518
Working on 2002: 9 of 94, date: 20020519
Working on 2002: 10 of 94, date: 20020520
Working on 2002: 11 of 94, date: 20020521
Working on 2002: 12 of 94, date: 20020522
Working on 2002: 13 of 94, date: 20020523
Working on 2002: 14 of 94, date: 20020525
Working on 2002: 15 of 94, date: 20020527
Working on 2002: 16 of 94, date: 20020528
Working on 2002: 17 of 94, date: 20020529
Working on 2002: 18 of 94, date: 20020530
Working on 2002: 19 of 94, date: 20020601
Working on 2002: 20 of 94, date: 20020602
Working on 2002: 21 of 94, date: 20020603
Working on 2002: 22 of 94, date: 20020604
Working on 2002: 23 of 94, date: 20020605
Working on 2002: 24 of 94, date: 20020606
W

Working on 2004: 10 of 93, date: 20040530
Working on 2004: 11 of 93, date: 20040601
Working on 2004: 12 of 93, date: 20040602
Working on 2004: 13 of 93, date: 20040603
Working on 2004: 14 of 93, date: 20040604
Working on 2004: 15 of 93, date: 20040605
Working on 2004: 16 of 93, date: 20040606
Working on 2004: 17 of 93, date: 20040609
Working on 2004: 18 of 93, date: 20040611
Working on 2004: 19 of 93, date: 20040612
Working on 2004: 20 of 93, date: 20040613
Working on 2004: 21 of 93, date: 20040614
Working on 2004: 22 of 93, date: 20040615
Working on 2004: 23 of 93, date: 20040616
Working on 2004: 24 of 93, date: 20040617
Working on 2004: 25 of 93, date: 20040618
Working on 2004: 26 of 93, date: 20040619
Working on 2004: 27 of 93, date: 20040620
Working on 2004: 28 of 93, date: 20040622
Working on 2004: 29 of 93, date: 20040623
Working on 2004: 30 of 93, date: 20040624
Working on 2004: 31 of 93, date: 20040625
Working on 2004: 32 of 93, date: 20040626
Working on 2004: 33 of 93, date: 2

Working on 2006: 11 of 92, date: 20060521
Working on 2006: 12 of 92, date: 20060523
Working on 2006: 13 of 92, date: 20060524
Working on 2006: 14 of 92, date: 20060525
Working on 2006: 15 of 92, date: 20060526
Working on 2006: 16 of 92, date: 20060527
Working on 2006: 17 of 92, date: 20060530
Working on 2006: 18 of 92, date: 20060531
Working on 2006: 19 of 92, date: 20060601
Working on 2006: 20 of 92, date: 20060602
Working on 2006: 21 of 92, date: 20060603
Working on 2006: 22 of 92, date: 20060604
Working on 2006: 23 of 92, date: 20060606
Working on 2006: 24 of 92, date: 20060607
Working on 2006: 25 of 92, date: 20060609
Working on 2006: 26 of 92, date: 20060610
Working on 2006: 27 of 92, date: 20060611
Working on 2006: 28 of 92, date: 20060613
Working on 2006: 29 of 92, date: 20060614
Working on 2006: 30 of 92, date: 20060615
Working on 2006: 31 of 92, date: 20060616
Working on 2006: 32 of 92, date: 20060617
Working on 2006: 33 of 92, date: 20060618
Working on 2006: 34 of 92, date: 2

Working on 2008: 16 of 103, date: 20080520
Working on 2008: 17 of 103, date: 20080521
Working on 2008: 18 of 103, date: 20080522
Working on 2008: 19 of 103, date: 20080523
Working on 2008: 20 of 103, date: 20080524
Working on 2008: 21 of 103, date: 20080525
Working on 2008: 22 of 103, date: 20080527
Working on 2008: 23 of 103, date: 20080529
Working on 2008: 24 of 103, date: 20080530
Working on 2008: 25 of 103, date: 20080531
Working on 2008: 26 of 103, date: 20080601
Working on 2008: 27 of 103, date: 20080603
Working on 2008: 28 of 103, date: 20080604
Working on 2008: 29 of 103, date: 20080606
Working on 2008: 30 of 103, date: 20080607
Working on 2008: 31 of 103, date: 20080608
Working on 2008: 32 of 103, date: 20080610
Working on 2008: 33 of 103, date: 20080611
Working on 2008: 34 of 103, date: 20080612
Working on 2008: 35 of 103, date: 20080613
Working on 2008: 36 of 103, date: 20080614
Working on 2008: 37 of 103, date: 20080615
Working on 2008: 38 of 103, date: 20080616
Working on 

Working on 2009: 95 of 102, date: 20090923
Working on 2009: 96 of 102, date: 20090925
Working on 2009: 97 of 102, date: 20090926
Working on 2009: 98 of 102, date: 20090929
Working on 2009: 99 of 102, date: 20091001
Working on 2009: 100 of 102, date: 20091004
Working on 2009: 101 of 102, date: 20091007
Working on 2009: 102 of 102, date: 20091009
Working on 2010: 1 of 96, date: 20100430
Working on 2010: 2 of 96, date: 20100501
Working on 2010: 3 of 96, date: 20100502
Working on 2010: 4 of 96, date: 20100504
Working on 2010: 5 of 96, date: 20100505
Working on 2010: 6 of 96, date: 20100506
Working on 2010: 7 of 96, date: 20100507
Working on 2010: 8 of 96, date: 20100508
Working on 2010: 9 of 96, date: 20100509
Working on 2010: 10 of 96, date: 20100510
Working on 2010: 11 of 96, date: 20100511
Working on 2010: 12 of 96, date: 20100515
Working on 2010: 13 of 96, date: 20100516
Working on 2010: 14 of 96, date: 20100519
Working on 2010: 15 of 96, date: 20100520
Working on 2010: 16 of 96, date:

Working on 2011: 93 of 95, date: 20111002
Working on 2011: 94 of 95, date: 20111005
Working on 2011: 95 of 95, date: 20111007
Working on 2012: 1 of 101, date: 20120505
Working on 2012: 2 of 101, date: 20120507
Working on 2012: 3 of 101, date: 20120508
Working on 2012: 4 of 101, date: 20120509
Working on 2012: 5 of 101, date: 20120510
Working on 2012: 6 of 101, date: 20120511
Working on 2012: 7 of 101, date: 20120513
Working on 2012: 8 of 101, date: 20120514
Working on 2012: 9 of 101, date: 20120515
Working on 2012: 10 of 101, date: 20120518
Working on 2012: 11 of 101, date: 20120519
Working on 2012: 12 of 101, date: 20120520
Working on 2012: 13 of 101, date: 20120522
Working on 2012: 14 of 101, date: 20120524
Working on 2012: 15 of 101, date: 20120525
Working on 2012: 16 of 101, date: 20120526
Working on 2012: 17 of 101, date: 20120527
Working on 2012: 18 of 101, date: 20120529
Working on 2012: 19 of 101, date: 20120530
Working on 2012: 20 of 101, date: 20120531
Working on 2012: 21 of 

Working on 2013: 88 of 107, date: 20130904
Working on 2013: 89 of 107, date: 20130906
Working on 2013: 90 of 107, date: 20130907
Working on 2013: 91 of 107, date: 20130908
Working on 2013: 92 of 107, date: 20130910
Working on 2013: 93 of 107, date: 20130911
Working on 2013: 94 of 107, date: 20130912
Working on 2013: 95 of 107, date: 20130913
Working on 2013: 96 of 107, date: 20130914
Working on 2013: 97 of 107, date: 20130915
Working on 2013: 98 of 107, date: 20130919
Working on 2013: 99 of 107, date: 20130920
Working on 2013: 100 of 107, date: 20130921
Working on 2013: 101 of 107, date: 20130922
Working on 2013: 102 of 107, date: 20130923
Working on 2013: 103 of 107, date: 20130926
Working on 2013: 104 of 107, date: 20130929
Working on 2013: 105 of 107, date: 20131006
Working on 2013: 106 of 107, date: 20131008
Working on 2013: 107 of 107, date: 20131010
Working on 2014: 1 of 99, date: 20140502
Working on 2014: 2 of 99, date: 20140504
Working on 2014: 3 of 99, date: 20140505
Working o

Working on 2015: 77 of 95, date: 20150908
Working on 2015: 78 of 95, date: 20150909
Working on 2015: 79 of 95, date: 20150911
Working on 2015: 80 of 95, date: 20150913
Working on 2015: 81 of 95, date: 20150917
Working on 2015: 82 of 95, date: 20150918
Working on 2015: 83 of 95, date: 20150919
Working on 2015: 84 of 95, date: 20150920
Working on 2015: 85 of 95, date: 20150921
Working on 2015: 86 of 95, date: 20150922
Working on 2015: 87 of 95, date: 20150923
Working on 2015: 88 of 95, date: 20150924
Working on 2015: 89 of 95, date: 20150927
Working on 2015: 90 of 95, date: 20150929
Working on 2015: 91 of 95, date: 20151004
Working on 2015: 92 of 95, date: 20151006
Working on 2015: 93 of 95, date: 20151009
Working on 2015: 94 of 95, date: 20151011
Working on 2015: 95 of 95, date: 20151014
Working on 2018: 1 of 89, date: 20180506
Working on 2018: 2 of 89, date: 20180507
Working on 2018: 3 of 89, date: 20180508
Working on 2018: 4 of 89, date: 20180511
Working on 2018: 5 of 89, date: 201805

Working on 2019: 87 of 101, date: 20190904
Working on 2019: 88 of 101, date: 20190905
Working on 2019: 89 of 101, date: 20190906
Working on 2019: 90 of 101, date: 20190908
Working on 2019: 91 of 101, date: 20190911
Working on 2019: 92 of 101, date: 20190915
Working on 2019: 93 of 101, date: 20190917
Working on 2019: 94 of 101, date: 20190919
Working on 2019: 95 of 101, date: 20190922
Working on 2019: 96 of 101, date: 20190924
Working on 2019: 97 of 101, date: 20190929
Working on 2019: 98 of 101, date: 20191001
Working on 2019: 99 of 101, date: 20191006
Working on 2019: 100 of 101, date: 20191008
Working on 2019: 101 of 101, date: 20191010
Working on 2020: 1 of 57, date: 20200725
Working on 2020: 2 of 57, date: 20200726
Working on 2020: 3 of 57, date: 20200728
Working on 2020: 4 of 57, date: 20200729
Working on 2020: 5 of 57, date: 20200730
Working on 2020: 6 of 57, date: 20200731
Working on 2020: 7 of 57, date: 20200801
Working on 2020: 8 of 57, date: 20200802
Working on 2020: 9 of 57,